<a href="https://colab.research.google.com/github/JonathanPhillips/2022-VOR-Model/blob/main/VOR2022v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
df = pd.read_csv('/content/all_compiled.csv', index_col=0)

comma_columns = ['RUSH_YD', 'REC_YD', 'PASS_YD']
for column in comma_columns:
  df[column] = df[column].str.replace(',', '').astype(float)

df['PPR'] = df['RUSH_YD'] + df['REC_YD'] + df['PASS_TD']*.04 + df['RUSH_TD'] + df['REC_TD']*6 \
+ df['PASS_TD']*4 + df['REC'] + df['FL']*-2 + df['INTS']*-2

df.sort_values(by='PPR', ascending=False).head()

In [20]:
ADP_URL = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'

res = requests.get(ADP_URL)

soup = BS(res.content, 'html.parser')

table = soup.find('table', attrs={
    'id': 'data'
})

adp_df = pd.read_html(str(table))[0]

adp_df['Player Team (Bye)'] = adp_df['Player Team (Bye)'].str.replace(' O', '')
adp_df['Player'] = adp_df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2]))
adp_df['Team'] = adp_df['Player Team (Bye)'].apply(lambda x: x.split()[-2])

adp_df.head()



,Rank,Player Team (Bye),POS,ESPN,RTSports,MFL,Fantrax,FFC,Sleeper,AVG,Player,Team
0,1,Christian McCaffrey CAR (13),RB1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Christian McCaffrey,CAR
1,2,Dalvin Cook MIN (7),RB2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Dalvin Cook,MIN
2,3,Alvin Kamara NO (6),RB3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,Alvin Kamara,NO
3,4,Derrick Henry TEN (13),RB4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Derrick Henry,TEN
4,5,Ezekiel Elliott DAL (7),RB5,8.0,5.0,6.0,5.0,5.0,5.0,5.8,Ezekiel Elliott,DAL


In [14]:
adp_df['POS'] = adp_df['POS'].str[:2]
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'Rank']]
adp_df.head()

,Player,Team,POS,Rank
0,Christian McCaffrey,CAR,RB,1
1,Dalvin Cook,MIN,RB,2
2,Alvin Kamara,NO,RB,3
3,Derrick Henry,TEN,RB,4
4,Ezekiel Elliott,DAL,RB,5


In [16]:
adp_cutoff = 150
adp_cutoff_df = adp_df.sort_values(by='Rank')[:adp_cutoff]
adp_cutoff_df = adp_cutoff_df.merge(df.loc[:, ['Player', 'Team', 'POS', 'PPR']], on=['Player', 'Team', 'POS'])
adp_cutoff_df.head(10)

,Player,Team,POS,Rank,PPR
0,Christian McCaffrey,CAR,RB,1,1665.2
1,Dalvin Cook,MIN,RB,2,1705.6
2,Alvin Kamara,NO,RB,3,1513.1
3,Derrick Henry,TEN,RB,4,1838.8
4,Ezekiel Elliott,DAL,RB,5,1149.3
5,Davante Adams,LV,WR,6,1509.4
6,Travis Kelce,KC,TE,7,1385.5
7,Aaron Jones,GB,RB,8,1350.6
8,Saquon Barkley,NYG,RB,9,1302.8
9,Austin Ekeler,LAC,RB,10,1745.5


In [17]:
replacement_values = {}
for _, row in adp_cutoff_df.iterrows():
  replacement_values[row['POS']] = row['PPR']

replacement_values

{'QB': 553.932, 'RB': 609.4000000000001, 'TE': 625.3, 'WR': 940.0999999999999}

In [18]:
vor_df = df.loc[:, ['Player', 'Team', 'POS', 'PPR']].rename({'POS': 'Position'}, axis=1)
vor_df['VOR'] = vor_df.apply(lambda row: row['PPR'] - replacement_values[row['Position']], axis=1)
vor_df.head()

,Player,Team,Position,PPR,VOR
1,Jonathan Taylor,IND,RB,2043.3,1433.9
2,Austin Ekeler,LAC,RB,1745.5,1136.1
3,Derrick Henry,TEN,RB,1838.8,1229.4
4,Dalvin Cook,MIN,RB,1705.6,1096.2
5,Joe Mixon,CIN,RB,1493.1,883.7


In [19]:
vor_df.sort_values(by='VOR', ascending=False).head(20)

,Player,Team,Position,PPR,VOR
1,Jonathan Taylor,IND,RB,2043.3,1433.9
3,Derrick Henry,TEN,RB,1838.8,1229.4
2,Austin Ekeler,LAC,RB,1745.5,1136.1
4,Dalvin Cook,MIN,RB,1705.6,1096.2
8,Christian McCaffrey,CAR,RB,1665.2,1055.8
7,Najee Harris,PIT,RB,1594.1,984.7
9,Alvin Kamara,NO,RB,1513.1,903.7
5,Joe Mixon,CIN,RB,1493.1,883.7
10,David Montgomery,CHI,RB,1487.3,877.9
1,Cooper Kupp,LAR,WR,1783.7,843.6
